# Step 1: fine-tune LLM using top result from (fixed) ranker

In [ ]:
def step_one(*, k, max_iteration):
    import evaluate
    import os
    from PersonalizedNews import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from peft import IA3Config, TaskType
    from TaskLLM import TaskLLM
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave, set_directory, GPUMonitor

    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    augment = int(os.environ.get('AUGMENT', '1'))
    output_dir = os.environ.get('AMLT_OUTPUT_DIR', '.')

    torch.manual_seed(2112)

    train = train_loader(batch_size=8*torch.cuda.device_count(), augment=augment)
    dev = dev_loader(batch_size=16)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    taskllm_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(taskllm_config, "taskllm")
    t5.enable_adapters()

    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rouge_metric = evaluate.load('rouge')

    def inner_batch(func, inner_batch_size, inputs):
        from more_itertools import chunked
        return [ func(*ib) for ib in zip(*[ chunked(g, inner_batch_size) for g in inputs ]) ]

    monitor = GPUMonitor(delay=60, maxcount=5)

    print(f'************ augment = {augment} *************')
    with ProgressPrinter('iter', f'{k} loss', f'{k} rouge1', f'{k} rouge1 (dev)') as printer:
        cumsum = lambda z, acc=0: [0] + [ acc := acc + v for v in z ]

        for iteration in range(max_iteration):
            for istrain, (examples, labels) in interleave(train, dev, sequential=True):
                with torch.no_grad():
                    texts_to_embed = [ [ ex['article'] ] + 
                                       [ v['text']
                                         for v in ex['profile']
                                       ]
                                       for ex in examples
                                     ]
                    embeddings = torch.cat(inner_batch(func = dev.embed,
                                                       inner_batch_size = 64 * torch.cuda.device_count(),
                                                       inputs = (sum(texts_to_embed, []),)
                                                      ),
                                           dim=0)
                    splits = cumsum(map(len, texts_to_embed))
                    indices = [ torch.topk(embeddings[a,:] @ embeddings[a+1:b,:].T, dim=0, k=k).indices for a, b in zip(splits, splits[1:]) ]
                    prompts = [ dev.prepend_to_prompt(ex, [ ex['profile'][ind] for ind in index.to('cpu').tolist() ])
                                for ex, index in zip(examples, indices) ]
                    guesses = sum(inner_batch(func = taskllm.generate,
                                              inner_batch_size = 32 * torch.cuda.device_count(),
                                              inputs = (prompts,)
                                             ),
                                  [])
                    scores = rouge_metric.compute(predictions=guesses, references=labels)['rouge1']

                if istrain:
                    losses = inner_batch(func = lambda p, t: (len(p), taskllm.learn(p, t)),
                                         inner_batch_size = 16 * torch.cuda.device_count(),
                                         inputs = (prompts, labels)
                                        )
                    loss = sum(w * l for w, l in losses) / sum(w for w, l in losses)
                else:
                    loss = None
                printer.addobs(iteration, loss, scores if istrain else None, scores if not istrain else None)

            printer.print()
            printer.autoprint = False
            with set_directory(output_dir):
                taskllm.save_pretrained(f'User_keq{k}_t5base_step1_iter{iteration}_augment{augment}')

step_one(k=4, max_iteration=5)

# Step 2: learn ranker using (fixed pre-finetuned) task LLM

In [ ]:
def learn_ranker(*, step1_iter, max_iteration, k):
    import evaluate
    from more_itertools import chunked
    from RewardPredictor import RewardPredictor
    from TaskLLM import TaskLLM
    from PersonalizedNews import train_loader, dev_loader
    from ProgressPrinter import ProgressPrinter
    from SimpleRegret import SimpleRegretHypercubeSampler
    from peft import IA3Config, TaskType
    from transformers import T5ForConditionalGeneration
    import torch
    from Util import interleave
    
    device = 'cuda'
    torch.set_default_device(device)
    torch.manual_seed(2112)

    train = train_loader(batch_size=8, augment=4)
    dev = dev_loader(batch_size=16)

    t5 = T5ForConditionalGeneration.from_pretrained('google/flan-t5-base')
    t5.load_adapter(f'User_keq{k}_t5base_step1_iter{step1_iter}', 'taskllm')

    rhat_config = IA3Config(task_type=TaskType.SEQ_2_SEQ_LM)
    t5.add_adapter(rhat_config, "rhat")
    t5.enable_adapters()
    
    taskllm = TaskLLM(t5=t5, adapter_name="taskllm")
    rewardpredictor = RewardPredictor(t5=t5, adapter_name="rhat")

    gumbel = torch.distributions.gumbel.Gumbel(0,1)
    def randomized_similarity(ex, nsamples):
        embeddings = dev.embed( [ ex['article'] ] + 
                                [ v['text']
                                  for v in ex['profile']
                                ])
        scores = embeddings[0,:] @ embeddings[1:,:].T
        temperature = scores[0].item() - scores[min(scores.shape[0], 4)].item()
        gumbel_shape = torch.Size([nsamples, scores.shape[0]])
        gumbels = temperature * gumbel.sample(gumbel_shape)
        return torch.unique(torch.topk(scores.unsqueeze(0) + gumbels, dim=1, k=k).indices, sorted=False, dim=0).to('cpu')

    rouge_metric = evaluate.load('rouge')

    with ProgressPrinter('iter', f'{k} loss', f'{k} rouge1', f'{k} rouge1 (dev)', 'samps') as printer:
        for iteration in range(max_iteration):
            for istrain, (examples, labels) in interleave(train, dev, sequential=True):
                greedyrewards, allloss, nsamps = [], [], []

                for ex, label in zip(examples, labels):
                    with torch.no_grad():
                        randos = randomized_similarity(ex, 64)
                        
                        prompts, rhatprompts = [], []
                        for rando in randos:
                            profile_examples = [ ex['profile'][ind] for ind in rando ]
                            prompt = dev.prepend_to_prompt(ex, profile_examples)
                            prompts.append(prompt)
                            rhatprompts.append(prompt)

                        rhats = rewardpredictor.predict(rhatprompts)
                        exploit, explore = SimpleRegretHypercubeSampler(rhats.view(1, -1), gamma=10)
                        exploit = [ exploit.item() ]
                        explore = explore[0].tolist() if istrain else []
                        actionind = exploit + [ n for n, observed in enumerate(explore) if observed > 0 ]
                        nsamps.append(len(actionind))
                        guesses = taskllm.generate([ prompts[a] for a in actionind ])
                        rewards = rouge_metric.compute(predictions=guesses,
                                                       references=[label]*len(guesses),
                                                       use_aggregator=False)['rouge1']
                        greedyrewards.append(rewards[0])

                    if istrain:
                        inner_batch_size = 4
                        loss = sum(
                            len(inner_batch[0]) * 
                            rewardpredictor.learn([ rhatprompts[a] for a in inner_batch[0] ], 
                                                    torch.Tensor([ [ r ] for r in inner_batch[1] ]).to(device)
                                                 )
                            for inner_batch in zip(chunked(actionind, inner_batch_size), chunked(rewards, inner_batch_size))
                        ) / len(actionind)
                        allloss.append(loss)

                greedyreward = torch.Tensor(greedyrewards).float().mean().item()
                predloss = torch.Tensor(allloss).mean().item() if istrain else None
                nsamps = torch.Tensor(nsamps).float().mean().item() if istrain else None

                printer.addobs(iteration, predloss, greedyreward if istrain else None, greedyreward if not istrain else None, nsamps)

            printer.print()
            printer.autoprint = False
            rewardpredictor.save_pretrained(f'User_keq{k}_t5base_step2_iter{iteration}')

learn_ranker(k=4, max_iteration=12, step1_iter=1)